# Load Layer outputs

In [1]:
nn = 'NN'

import pickle
with open('results/' +nn+'_layer_outputs.dat','rb') as f:
    layer_outs,layer_outs_test= pickle.load(f)
f.close()

# Load recording and test data

In [2]:
import numpy as np
from keras.datasets import mnist
from scipy.misc import imresize

(Xtrain, Ytrain), (Xtest, Ytest) = mnist.load_data()

(ntrain, xdim, ydim) = Xtrain.shape
ntest = Xtest.shape[0]

# Recording data
X_pr = Xtrain[30000:60000, :, :]
Y_pr = Ytrain[30000:60000]

# downsample
factor = 0.25

Xtest_down = np.ones((Xtest.shape[0], int(xdim*factor), int(ydim*factor)))
for i in range(Xtest.shape[0]):
    Xtest_down[i,:,:] = imresize(Xtest[i,:,:], factor)

X_pr_down = np.ones((X_pr.shape[0], int(xdim*factor), int(ydim*factor)))
for i in range(X_pr.shape[0]):
    X_pr_down[i,:,:] = imresize(X_pr[i,:,:], factor)
    
# VECTORIZE IMAGES
Xtest_down = Xtest_down.reshape(ntest, int(xdim*factor)**2).astype('float32') / 255
X_pr_down = X_pr_down.reshape(X_pr_down.shape[0], int(xdim*factor)**2).astype('float32') / 255

Using TensorFlow backend.


# Run Neural Networks and average outputs.

In [ ]:
from copy import copy, deepcopy
from RE_PartialRecData2 import RE_PartialRecData2
from sklearn import preprocessing, linear_model
import pickle
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import np_utils
from keras.models import load_model
from keras import backend as K
import tensorflow as tf

# how many recordings?
nRecordings = 10
# how many neurons from the firs hidden layer?
subnetSize = [2**x for x in range(8)]
#subnetSize =[1]
nSubnetSize = len(subnetSize)
# which layers?
iLayer=0
# how many samples per recording?
nSamples = np.divide(int(X_pr_down.shape[0]/nRecordings),subnetSize)*100
# how many iterations
nIterations = 5

# baseline prediction error
#bl = np.std(layer_outs_test[oLayer]-np.mean(layer_outs_test[oLayer]));

oLayer = len(layer_outs)-1  # index of output layer
nOutNeurons = layer_outs[oLayer].shape[1]
rmses = np.zeros([nIterations, nOutNeurons, nSubnetSize])

for iLayer in range(0, len(layer_outs)-1,2):
    print('Layer #', iLayer)
    for ss in range(nSubnetSize):
        if subnetSize[ss]<16:
            continue
        #r_coefs =np.zeros((nIterations, 10, 128))
        #nLayerNeurons = [subnetSize[ss], 0, 0, 0, 10]
        nLayerNeurons = subnetSize[ss]
        print(subnetSize[ss])
        for it in range(nIterations):
            # copy data - is this necessary?
            layer_outputs = deepcopy(layer_outs)
            rmses_rec = np.zeros((nRecordings, nOutNeurons))
            for iRec in range(nRecordings):
                # subsample
                X_subsample, Y_subsample = RE_PartialRecData2(layer_outputs[iLayer], layer_outputs[oLayer], \
                                                              nLayerNeurons, 1, nSamples[ss])
                # impute X_subsample with mean value, apply imputation to test set
                imp =preprocessing.Imputer(missing_values='NaN', strategy='mean')
                impf =imp.fit(X_subsample)
                X_new = impf.transform(X_subsample)
                (rem_cols, ) = np.where(np.sum(np.abs(X_new), axis=0)==0)
                if rem_cols.shape[0]>0:
                    print('Removing cols #', rem_cols)
                X_test_new= impf.transform(layer_outs_test[iLayer])
                # fit ΝΝ
                model = Sequential()
                model.add(Dense(50, input_shape=(X_new.shape[1],), activation='tanh'))
                model.add(Dense(50, activation='tanh'))
                model.add(Dense(10, activation='tanh'))
                # Compile model
                model.compile(loss='mean_squared_error', optimizer='adam')
                model.fit(X_new, Y_subsample, nb_epoch=20, batch_size =16, verbose=0)

                rmses_rec[iRec, :] = np.sqrt(np.mean((model.predict(X_test_new) - layer_outs_test[oLayer])**2, axis=0))
            #print(rmses_rec)

            rmses[it,:,ss] = np.mean(rmses_rec, axis=0)

            print ('predicting, mean RMSEs=%f' %np.mean(rmses[it, :, ss]))

    fName = 'results/NNMP_RMSES_Layer'+str(iLayer) + 'nRec' + str(nRecordings) + nn + '.dat'
    # save the rmse's
    with open(fName,'wb') as f:
        pickle.dump(rmses, f)
    f.close()

('Layer #', 0)
16
('Removing cols #', array([9]))
Epoch 1/20
3s - loss: 0.0553
Epoch 2/20
4s - loss: 0.0523
Epoch 3/20
4s - loss: 0.0499
Epoch 4/20
3s - loss: 0.0467
Epoch 5/20
3s - loss: 0.0440
Epoch 6/20
3s - loss: 0.0425
Epoch 7/20
3s - loss: 0.0413
Epoch 8/20
4s - loss: 0.0403
Epoch 9/20
4s - loss: 0.0395
Epoch 10/20
3s - loss: 0.0388
Epoch 11/20
3s - loss: 0.0383
Epoch 12/20
3s - loss: 0.0377
Epoch 13/20
3s - loss: 0.0372
Epoch 14/20
3s - loss: 0.0367
Epoch 15/20
3s - loss: 0.0364
Epoch 16/20
3s - loss: 0.0360
Epoch 17/20
3s - loss: 0.0357
Epoch 18/20
3s - loss: 0.0354
Epoch 19/20
3s - loss: 0.0351
Epoch 20/20
4s - loss: 0.0348
Epoch 1/20
4s - loss: 0.0574
Epoch 2/20
4s - loss: 0.0548
Epoch 3/20
3s - loss: 0.0515
Epoch 4/20
4s - loss: 0.0463
Epoch 5/20
4s - loss: 0.0422
Epoch 6/20
4s - loss: 0.0398
Epoch 7/20
3s - loss: 0.0379
Epoch 8/20
4s - loss: 0.0364
Epoch 9/20
3s - loss: 0.0353
Epoch 10/20
3s - loss: 0.0345
Epoch 11/20
4s - loss: 0.0337
Epoch 12/20
4s - loss: 0.0329
Epoch 13

In [ ]:
model